In [ ]:
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
from apiclient.discovery import build

In [ ]:
youtube = build('youtube','v3', developerKey=api_key)

In [ ]:
res = youtube.channels().list(id='UCXIJgqnII2ZOINSWNOGFThA',
                              part='contentDetails').execute()

In [ ]:
res

In [ ]:
res = youtube.playlistItems().list(playlistId = 'UUXIJgqnII2ZOINSWNOGFThA',
                                   part='snippet',
                                  maxResults=50).execute()


In [ ]:
res 

In [ ]:
def get_channel_videos(channel_id):
    
    # get Uploads playlist id
    res = youtube.channels().list(id=channel_id, 
                                  part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                           part='snippet', 
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')
    
        if next_page_token is None:
            break
    
    return videos



In [ ]:
res["items"]



In [ ]:
videos = get_channel_videos('UCXIJgqnII2ZOINSWNOGFThA')


In [ ]:
len(videos)
videos[1]

In [ ]:
for video in videos:
    print(video['snippet']['title'])
    print(video['snippet']['publishedAt'])
   

In [ ]:
import pandas as pd
import io

In [ ]:
print(type(video['snippet']['title']))


In [ ]:
#Create the dataframe of titles and times
l = []
for video in videos:
    l.append((video['snippet']['title'], video['snippet']['publishedAt'],video['snippet'][ 'videoOwnerChannelTitle']))
l
df = pd.DataFrame(l, columns = ['Title', 'Time','Channel'])
df


In [ ]:
#Cleaning titles
import re

# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
df['Cleaned Title'] = df['Title'].apply(clean)
df.head()

In [ ]:
#nltk imports
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')



In [ ]:
#POS dictionary 
pos_dictionary = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dictionary.get(tag[0])]))
    return newlist

df['Tagged Words'] = df['Cleaned Title'].apply(token_stop_pos)
df.head()

In [ ]:
#obtaining the stem - lemmatization 
from nltk.corpus.reader.wordnet import NOUN
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew



In [ ]:
df['Lemma'] = df['Tagged Words'].apply(lemmatize)


In [ ]:
df

In [ ]:
from textblob import TextBlob

In [ ]:
# function to calculate subjectivity
def getSubjectivity(title):
    return TextBlob(title).sentiment.subjectivity

# function to calculate polarity
def getPolarity(title):
    return TextBlob(title).sentiment.polarity

In [ ]:
textblob_df = pd.DataFrame(df[['Title','Lemma']])

In [ ]:
textblob_df['Subjectivity'] = textblob_df['Lemma'].apply(getSubjectivity)
textblob_df['Polarity'] = textblob_df['Lemma'].apply(getPolarity)

In [ ]:
textblob_df